In [ ]:
#https://github.com/MIT-LCP/mimic-code/blob/master/concepts/severityscores/oasis.sql

In [ ]:
#Leaves out age and MV (which is already in chart_events)

In [ ]:
import os, re
import pandas as pd
import numpy as np
import time
from datetime import datetime

In [ ]:
#Comparison with what we have 
#item_file='M:/Raw_CSV/D_ITEMS.csv'
item_df = pd.read_csv("data-raw/mimic/D_ITEMS.csv")

In [ ]:
%%time

#import dask.dataframe as dd

cache_file = "data/cache/chartevents.csv"
if not os.path.exists(cache_file):
    # dask version is slower for some reason.
    #orig_df = dd.read_csv("data-raw/mimic/CHARTEVENTS.csv",
    # Takes about 22 mins to import with pandas.
    orig_df = pd.read_csv("data-raw/mimic/CHARTEVENTS.csv",
                 #nrows = 1000,
                 dtype = {'ROW_ID': 'UInt64',
                          'SUBJECT_ID': 'UInt64',
                          'HADM_ID': 'UInt64',
                          'ICUSTAYID': 'UInt64',
                          'ICUSTAY_ID':'UInt64', # This one needed, not sure about above row.
                          'ITEMID': 'UInt64',
                          'CGID': 'UInt64', 
                          'RESULTSTATUS': 'object',
                          'STOPPED': 'object',
                          'ERROR': 'UInt64',
                          'WARNING': 'UInt64',
                          'VALUE': 'object'})#,
                      #assume_missing = True)
    #orig_df = orig_df.compute()
    orig_df.to_feather(cache_file)
else:
    # Takes 2 mins to load - huge speedup.
    orig_df = pd.read_feather(cache_file)

In [ ]:
#Comparison with what we have 
"""
chart_file='M:/MIMIC_001_chartevents.csv'
pwd=os.getcwd()
os.chdir(os.path.dirname(chart_file))
chart_pts=pd.read_csv(os.path.basename(chart_file))
os.chdir(pwd)
"""
# Created by MIMIC_001_extract_outcomes.ipynb
chart_pts = pd.read_csv("data/MIMIC_001_chartevents.csv",
                       low_memory = False)

In [ ]:
"""
icu_file='M:/RAW_CSV/ICUSTAYS.csv'
pwd=os.getcwd()
os.chdir(os.path.dirname(icu_file))
icu_stay=pd.read_csv(os.path.basename(icu_file))
os.chdir(pwd)
"""
icu_stay = pd.read_csv("data-raw/mimic/ICUSTAYS.csv",
             usecols = ['SUBJECT_ID', 'HADM_ID', 'INTIME', 'OUTTIME', 'LOS'])

In [ ]:
#icu_stay.head(5)
#icu_stay=icu_stay[['SUBJECT_ID', 'HADM_ID', 'INTIME', 'OUTTIME', 'LOS']]

In [ ]:
#Comparison with what we have 
"""
df_file='M:/Raw_CSV/OUTPUTEVENTS.csv'
pwd=os.getcwd()
os.chdir(os.path.dirname(df_file))
out_df=pd.read_csv(os.path.basename(df_file))
os.chdir(pwd)
"""
out_df = pd.read_csv("data-raw/mimic/OUTPUTEVENTS.csv")

In [ ]:
#Comparison with what we have 
"""
admit_file='M:/RAW_CSV/ADMISSIONS.csv'
pwd=os.getcwd()
os.chdir(os.path.dirname(admit_file))
admit0=pd.read_csv(os.path.basename(admit_file))
os.chdir(pwd)
"""
admit0 = pd.read_csv("data-raw/mimic/ADMISSIONS.csv")

In [ ]:
admit0['elective_adm'] = admit0.ADMISSION_TYPE.map(lambda x: 1 if x=='ELECTIVE' else 0)
admit_df = admit0[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'elective_adm']]
admit_df.head(5)

In [ ]:
item_df.LABEL.unique().tolist()[:5]

In [ ]:
item_df.LABEL=item_df.LABEL.map(lambda x: str(x).lower())
def find_item(string): 
    if 'gcs total' in string: return 'GCS' 
    elif string=='heart rate': return 'HR'
    elif string=='map': return 'MAP'
    elif string=='aterial bp mean': return 'MAP'
    elif string=='nbp mean': return 'MAP'
    elif string=='manual bp mean(calc)': return 'MAP'
    elif '[temperature]' in string: return 'temp'
    elif '[temp]' in string: return 'temp'
    elif string=='respiratory rate': return 'RR'
    elif 'urine out' in string: return 'uop'
    else: return 0
    
item_df['Oasis'] = item_df.LABEL.map(lambda x: find_item(x))
oasis_df = item_df[item_df.Oasis!=0]
oasis_items = oasis_df[['ITEMID', 'LABEL', 'Oasis']].ITEMID.tolist()
print(len(oasis_items))

In [ ]:
def find_oasis_item(integer, item_list): 
    if integer in item_list: return 1
    else: return 0

In [ ]:
%%time
# This is very slow.
orig_df['oasis_ITEM'] = orig_df.ITEMID.map(lambda x: find_oasis_item(int(x), oasis_items))

In [ ]:
chart_df0 = orig_df[orig_df.oasis_ITEM == 1]
chart_df = chart_df0[['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'VALUE']]

In [ ]:
chart_df.head(3)

In [ ]:
##insert merged dfs here

In [ ]:
chart_events = pd.merge(chart_df, oasis_df[['ITEMID', 'Oasis']], on='ITEMID', how='left')
chart_events['Oasis_Value'] = pd.to_numeric(chart_events['VALUE'], errors='coerce')

In [ ]:
uop_df0 = out_df[['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'VALUE']]
uop_df0['oasis_ITEM'] = uop_df0.ITEMID.map(lambda x: find_oasis_item(int(x), oasis_items))
uop_df = uop_df0[uop_df0.oasis_ITEM == 1]
uop_events = pd.merge(uop_df, oasis_df[['ITEMID', 'Oasis']], on='ITEMID', how='left')
uop_events['Oasis_Value'] = pd.to_numeric(uop_events['VALUE'], errors='coerce')
print(uop_df.shape)
uop_events.head(5)
uop_final = uop_events.groupby(['SUBJECT_ID', 'HADM_ID'], as_index=False)['Oasis_Value'].sum()
uop_final['total_uop'] = uop_final['Oasis_Value']
uop_final.pop('Oasis_Value')
print(uop_final.shape)

In [ ]:
chart_events.info()

In [ ]:
# Need to convert types to allow the merge to succeed.
uop_final.HADM_ID = uop_final.HADM_ID.astype('UInt64')

In [ ]:
# TODO: fix.
all_events = pd.merge(chart_events, uop_final, on=['SUBJECT_ID', 'HADM_ID'], how='left')
all_events.head(3)

In [ ]:
icu_admit = pd.merge(icu_stay, admit_df, on=['SUBJECT_ID', 'HADM_ID'], how='left')
icu_admit['admit_date'] = pd.to_datetime(icu_admit.ADMITTIME).dt.date
icu_admit['icu_admit'] = pd.to_datetime(icu_admit.INTIME).dt.date
icu_admit['pre_icu_los_days'] = icu_admit['icu_admit']-icu_admit['admit_date']
icu_admit['pre_icu_los'] = icu_admit['pre_icu_los_days'].map(lambda x: int(str(x)[0]) if str(x)[0].isnumeric() else 0 )
icu_admit = icu_admit[['SUBJECT_ID', 'HADM_ID', 'elective_adm', 'pre_icu_los']]

In [ ]:
events_admit = pd.merge(all_events, icu_admit, on=['SUBJECT_ID', 'HADM_ID'], how='left')

In [ ]:
events_admit

In [ ]:
events_admit.Oasis.value_counts()

In [ ]:
events_admit.groupby(['SUBJECT_ID', 'HADM_ID', 'elective_adm', 'pre_icu_los', 'total_uop', 'Oasis'])['Oasis_Value'].agg(max_val='max', min_val='min')

In [ ]:
#events_admit.to_csv('M:/MIMIC_001_OASIS_raw.csv', index=False)
events_admit.to_csv('data/MIMIC_001_OASIS_raw.csv', index=False)